In [174]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Install Keras Tuner

In [175]:
 !pip install Keras_tuner -qq

# Loading data

Data directory

In [176]:
train_dir = r"/content/gdrive/My Drive/Deep Learning/Datasets/dogs_vs_cats/train/valid"

In [177]:
import os, random
os.listdir(train_dir)

['dog', 'cat']

Data Labels

In [178]:
categories = ['dog', 'cat']

In [179]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
 
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

Extract data from directory

In [180]:
train_data = []
 
def create_training_data(directory,categories, data):
    for category in categories:
        path = os.path.join(directory,category)
        class_num = categories.index(category)
        
        for img in os.listdir(path):
            try:
                img_array = load_img(os.path.join(path,img), target_size=(100,100))
                new_array = img_to_array(img_array)
                data.append([new_array, class_num])
            except Exception as e:
                pass
    random.shuffle([train_data])

In [181]:
%%time
create_training_data(directory= train_dir,
                     categories=categories, data=train_data)

CPU times: user 467 ms, sys: 23.7 ms, total: 491 ms
Wall time: 640 ms


In [182]:
from sklearn.model_selection import train_test_split

In [183]:
len(train_data)

200

In [184]:
 def get_features_and_labels(data, features, labels):
    for feature, label in data:
        features.append(feature)
        labels.append(label)
    return features, labels

In [185]:
images, labels = [], []
images, labels = get_features_and_labels(train_data, images, labels)

In [186]:
print(len(images), len(labels))

200 200


Validation set

In [187]:
X_train, X_valid, y_train, y_valid = train_test_split(images, labels, test_size =0.3, random_state = 42)

In [188]:
X_train = np.array(X_train)
X_valid = np.array(X_valid)
 
y_train = np.array(y_train)
y_valid = np.array(y_valid)

Scale data

In [189]:
 def rescale(data):
    data = data/255
    return data
 
X_train = rescale(X_train)
X_valid = rescale(X_valid)

In [190]:
X_train[0].shape

(100, 100, 3)

Reshape image data

In [191]:
X_train = X_train.reshape(len(X_train), 100,100,3)
X_valid = X_valid.reshape(len(X_valid), 100,100,3)

# Build model
* a function as parameter for keras tuner

In [192]:
 def build_model(hp):
 
  model = keras.Sequential([
 
    keras.layers.Conv2D(
      filters=hp.Int("conv_1_filter", min_value=32, max_value=64, step=16),
      kernel_size=hp.Choice("conv_1_kernel", values = [2,5]),
      activation="relu",
      input_shape=(100, 100, 3),
      padding = 'same'
    ),
 
    keras.layers.Conv2D(
      filters =hp.Int("conv_2_filter", min_value= 64, max_value=96, step=16),
      kernel_size=hp.Choice("conv_2_kernel", values = [3,7]),
      activation = "relu",
      padding = 'same'
    ),
 
    keras.layers.Flatten(),
 
    keras.layers.Dense(
      units=hp.Int("dense_1_units", min_value=72, max_value=128, step=16),
      activation = 'relu'
    ),
 
    keras.layers.Dense(1, activation = "sigmoid")
    ])
 
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice("learning_rate",
                                                            values= [1e-2, 1e-3, 1e-4])),
      loss= 'binary_crossentropy',
      metrics=["accuracy"])
  return model

Import keras tuner and its

In [193]:
from kerastuner import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters

RandomSearch

In [194]:
tuner_search = RandomSearch(build_model,
objective = "val_accuracy",
max_trials = 5,
executions_per_trial= 1,
directory = 'output',
overwrite = True,project_name = 'Keras_tuner on Dogs_vs_Cats')

In [195]:
tuner_search.search(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 00m 18s]
val_accuracy: 0.5277777711550394

Best val_accuracy So Far: 0.5388888716697693
Total elapsed time: 00h 03m 13s
INFO:tensorflow:Oracle triggered exit


In [196]:
# , and Sklearn

# Hyperband

In [197]:
tuner_search_band = Hyperband(build_model,
objective = "val_accuracy",
max_epochs = 3,
hyperband_iterations = 2,
executions_per_trial= 1,
directory = 'output',
overwrite = True,
project_name = 'Keras_tuner on Dogs_vs_Cats_2')

In [ ]:
tuner_search_band.search(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 00m 51s]
val_accuracy: 0.5166666507720947

Best val_accuracy So Far: 0.5333333611488342
Total elapsed time: 00h 01m 54s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |32                |32                
conv_1_kernel     |5                 |2                 
conv_2_filter     |96                |64                
conv_2_kernel     |3                 |3                 
dense_1_units     |72                |72                
learning_rate     |0.001             |0.01              
tuner/epochs      |3                 |1                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |0                 |1                 
tuner/round       |0                 |0                 

Epoch 1/3
3/5 [=================>............] - ETA: 0s - loss: 22.6299 - accuracy: 0.4375

# BayesianOptimization

In [ ]:
tuner_search_bayes = BayesianOptimization(build_model,
objective = "val_accuracy",
num_initial_points =5,
max_trials = 5,
directory = 'output',
overwrite = True,
project_name = 'Keras_tuner on Dogs_vs_Cats_3')

In [ ]:
tuner_search_bayes.search(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))